# Exploring the cubing_algs VCube Class

This notebook demonstrates the `VCube` class from the `cubing_algs` library, designed for developers and speedcubing hobbyists who want to understand virtual cube simulation, state tracking, and visualization.

## What is a VCube?

A **VCube** (Virtual Cube) is a digital representation of a 3x3 Rubik's cube that can track the state of all 54 facelets, apply moves, and provide visual feedback. It's the foundation for cube simulation, algorithm testing, and state analysis.

Let's start by importing the necessary modules:

In [1]:
from cubing_algs.vcube import VCube
from cubing_algs.algorithm import Algorithm
from cubing_algs.move import Move
from cubing_algs.constants import FACE_ORDER, INITIAL_STATE

print("=== Creating VCube Objects ===")

# Method 1: Default solved cube
solved_cube = VCube()
print(f"Solved cube: {solved_cube.state}")
print(f"Is solved: {solved_cube.is_solved}")

# Method 2: From specific state
custom_state = "UUFUUFLLFUUURRRRRRFFRFFDFFDRRBDDBDDBLLDLLDLLDLBBUBBUBB"
custom_cube = VCube(custom_state)
print(f"\nCustom cube: {custom_cube.state}")
print(f"Is solved: {custom_cube.is_solved}")

# Method 3: Copy an existing cube
copied_cube = solved_cube.copy()
print(f"\nCopied cube: {copied_cube.state}")
print(f"Same as original: {copied_cube.state == solved_cube.state}")

print("\n=== Cube Properties ===")
print(f"Cube size: {solved_cube.size}x{solved_cube.size}")
print(f"Number of faces: {solved_cube.face_number}")
print(f"Face size: {solved_cube.face_size} facelets")
print(f"Total facelets: {len(solved_cube.state)}")
print(f"Face order: {FACE_ORDER}")

print("\nVCube objects created successfully!")

=== Creating VCube Objects ===
Solved cube: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Is solved: True

Custom cube: UUFUUFLLFUUURRRRRRFFRFFDFFDRRBDDBDDBLLDLLDLLDLBBUBBUBB
Is solved: False

Copied cube: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Same as original: True

=== Cube Properties ===
Cube size: 3x3
Number of faces: 6
Face size: 9 facelets
Total facelets: 54
Face order: ['U', 'R', 'F', 'D', 'L', 'B']

VCube objects created successfully!


## Understanding Cube State Representation

The VCube represents a cube's state as a 54-character string, where each character represents a facelet's color. The facelets are ordered by face (U, R, F, D, L, B, following the Kociemba's order) and within each face from top-left to bottom-right:

In [2]:
print("=== Cube State Structure ===")
print()
print("The 54-character state string represents facelets in this order:")
print("Faces: U (Up), R (Right), F (Front), D (Down), L (Left), B (Back)")
print("Each face has 9 facelets numbered 0-8:")
print()
print("Face layout (view from outside the cube):")
print("┌───────┐")
print("│ 0 1 2 │")
print("│ 3 4 5 │  ← 4 is the center")
print("│ 6 7 8 │")
print("└───────┘")
print()

def analyze_cube_state(cube, name):
    """Analyze and display cube state breakdown"""
    print(f"=== {name} State Analysis ===")
    print(f"Full state: {cube.state}")
    print()
    
    # Show each face
    for i, face in enumerate(FACE_ORDER):
        face_state = cube.get_face(face)
        print(f"{face} face: {face_state}")
        
        # Show 3x3 layout for the face
        print(f"  Layout: {face_state[0]}{face_state[1]}{face_state[2]}")
        print(f"          {face_state[3]}{face_state[4]}{face_state[5]}")
        print(f"          {face_state[6]}{face_state[7]}{face_state[8]}")
        print()
    
    # Show centers
    centers = cube.get_face_center_indexes()
    print(f"Face centers: {centers}")
    print(f"Orientation: {cube.orientation} (top-front)")
    print()

# Analyze solved cube
analyze_cube_state(solved_cube, "Solved Cube")

# Create a scrambled cube for comparison
scrambled_cube = VCube()
scrambled_cube.rotate("R U R' U' R U R' F' R U R' U' R' F R2 U' R'")
print(f"After T-Perm scramble: {scrambled_cube.state}")
print(f"Is solved: {scrambled_cube.is_solved}")

# Show just the changes
print("\nChanges from solved state:")
for i, (original, scrambled) in enumerate(zip(solved_cube.state, scrambled_cube.state)):
    if original != scrambled:
        face_idx = i // 9
        facelet_idx = i % 9
        face_name = FACE_ORDER[face_idx]
        print(f"  Position {i} ({face_name}{facelet_idx}): {original} → {scrambled}")

=== Cube State Structure ===

The 54-character state string represents facelets in this order:
Faces: U (Up), R (Right), F (Front), D (Down), L (Left), B (Back)
Each face has 9 facelets numbered 0-8:

Face layout (view from outside the cube):
┌───────┐
│ 0 1 2 │
│ 3 4 5 │  ← 4 is the center
│ 6 7 8 │
└───────┘

=== Solved Cube State Analysis ===
Full state: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB

U face: UUUUUUUUU
  Layout: UUU
          UUU
          UUU

R face: RRRRRRRRR
  Layout: RRR
          RRR
          RRR

F face: FFFFFFFFF
  Layout: FFF
          FFF
          FFF

D face: DDDDDDDDD
  Layout: DDD
          DDD
          DDD

L face: LLLLLLLLL
  Layout: LLL
          LLL
          LLL

B face: BBBBBBBBB
  Layout: BBB
          BBB
          BBB

Face centers: ['U', 'R', 'F', 'D', 'L', 'B']
Orientation: UF (top-front)

After T-Perm scramble: UUUFUULUFRRRBRRURRBFDFFUFFFDDRDDDDDDFRULLLLLLBLLBBBBBB
Is solved: False

Changes from solved state:
  Position 3 (U3): U 

## Applying Moves to the Cube

The core functionality of VCube is its ability to apply moves and track state changes. The cube supports all standard Rubik's cube notation:

In [3]:
print("=== Applying Moves to VCube ===")

def demonstrate_moves(move_sequence, description):
    """Demonstrate applying moves to a cube"""
    cube = VCube()
    print(f"\n{description}:")
    print(f"Moves: {move_sequence}")
    print(f"Before: {cube.state}")
    print(f"Is solved before: {cube.is_solved}")
    
    # Apply moves
    cube.rotate(move_sequence)
    
    print(f"After:  {cube.state}")
    print(f"Is solved after: {cube.is_solved}")
    
    return cube

# Test different types of moves
basic_moves = [
    ("R", "Single clockwise R turn"),
    ("R'", "Single counter-clockwise R turn"),
    ("R2", "Double R turn (180°)"),
    ("U R U' R'", "Sexy move sequence"),
    ("M E S", "Slice moves"),
    ("x y z", "Cube rotations"),
    ("Rw Uw Fw", "Wide moves"),
]

results = []
for moves, description in basic_moves:
    result_cube = demonstrate_moves(moves, description)
    results.append((moves, result_cube))

print("\n=== Move History Tracking ===")
cube_with_history = VCube()
print(f"Initial history: {cube_with_history.history}")

# Apply moves and track history
cube_with_history.rotate("R U R' U'")
print(f"After 'R U R\' U\'': {cube_with_history.history}")

cube_with_history.rotate("F R F'")
print(f"After 'F R F\'': {cube_with_history.history}")

# Apply moves without history
cube_with_history.rotate("D L D'", history=False)
print(f"After 'D L D\' (no history)': {cube_with_history.history}")

print("\n=== Using Algorithm Objects ===")
alg_cube = VCube()
t_perm = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R'")

print(f"Applying algorithm: {t_perm}")
print(f"Before: Is solved = {alg_cube.is_solved}")

alg_cube.rotate(t_perm)
print(f"After: Is solved = {alg_cube.is_solved}")
print(f"History length: {len(alg_cube.history)}")
print(f"History: {' '.join(alg_cube.history)}")

=== Applying Moves to VCube ===

Single clockwise R turn:
Moves: R
Before: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Is solved before: True
After:  UUFUUFUUFRRRRRRRRRFFDFFDFFDDDBDDBDDBLLLLLLLLLUBBUBBUBB
Is solved after: False

Single counter-clockwise R turn:
Moves: R'
Before: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Is solved before: True
After:  UUBUUBUUBRRRRRRRRRFFUFFUFFUDDFDDFDDFLLLLLLLLLDBBDBBDBB
Is solved after: False

Double R turn (180°):
Moves: R2
Before: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Is solved before: True
After:  UUDUUDUUDRRRRRRRRRFFBFFBFFBDDUDDUDDULLLLLLLLLFBBFBBFBB
Is solved after: False

Sexy move sequence:
Moves: U R U' R'
Before: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Is solved before: True
After:  RFUUUUUURDBBRRRRRRFFFFFUFFUDDFDDDDDDULLLLLLLLLRBBBBBBB
Is solved after: False

Slice moves:
Moves: M E S
Before: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Is solved before: True
After:  UBULDLUBU

## Cube Visualization and Display

VCube provides powerful visualization capabilities to display the cube state in various formats and orientations:

In [4]:
print("=== Cube Visualization ===")

# Create some interesting cube states
cubes_to_visualize = []

# Solved cube
solved = VCube()
cubes_to_visualize.append(("Solved Cube", solved))

# Sexy move applied
sexy_cube = VCube()
sexy_cube.rotate("R U R' U'")
cubes_to_visualize.append(("After Sexy Move", sexy_cube))

# Cross pattern (simplified)
cross_cube = VCube()
cross_cube.rotate("F R U R' U' F'")
cubes_to_visualize.append(("After F R U R' U' F'", cross_cube))

# T-Perm state
t_perm_cube = VCube()
t_perm_cube.rotate("R U R' F' R U R' U' R' F R2 U' R'")
cubes_to_visualize.append(("After T-Perm", t_perm_cube))

print("Basic cube visualization using __str__ method:\n")
for name, cube in cubes_to_visualize:
    print(f"=== {name} ===")
    print(cube)
    print(f"State string: {cube.state}")
    print(f"Solved: {cube.is_solved}")
    print()

print("\n=== Advanced Display Options ===")
print("The VCube class supports various display modes and options:")
print("• show() method - prints visual representation")
print("• display() method - returns visual representation as string")
print("• Support for different modes, orientations, and color palettes")
print("• Masking capabilities to highlight specific parts")
print("• Effects and custom facelet representations")

# Demonstrate basic display
demo_cube = VCube()
demo_cube.rotate("R U R' U'")

print(f"\nDemo cube after 'R U R\' U\'':")
try:
    # This will show the cube if display system is available
    demo_cube.show()
    print("\n(Visual display shown above)")
except Exception as e:
    print(f"Visual display not available: {e}")
    print("Fallback to state string:")
    print(demo_cube)

print("\n=== Face Access Methods ===")
demo_cube = VCube()
demo_cube.rotate("R U2 F'")

print("Individual face access:")
for face in FACE_ORDER:
    face_state = demo_cube.get_face(face)
    print(f"{face} face: {face_state}")

print("\nFace centers:")
centers = demo_cube.get_face_center_indexes()
for i, (face, center) in enumerate(zip(FACE_ORDER, centers)):
    print(f"{face} center: {center}")

print(f"\nCube orientation: {demo_cube.orientation}")

# Demonstrate face access by center color
print("\nAccess face by center color:")
try:
    u_face_by_center = demo_cube.get_face_by_center('U')
    print(f"Face with U center: {u_face_by_center}")
    
    f_face_by_center = demo_cube.get_face_by_center('F')
    print(f"Face with F center: {f_face_by_center}")
except Exception as e:
    print(f"Error accessing face by center: {e}")

=== Cube Visualization ===
Basic cube visualization using __str__ method:

=== Solved Cube ===
U: UUUUUUUUU
R: RRRRRRRRR
F: FFFFFFFFF
D: DDDDDDDDD
L: LLLLLLLLL
B: BBBBBBBBB
State string: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
Solved: True

=== After Sexy Move ===
U: UULUUFUUF
R: RRUBRRURR
F: FFDFFUFFF
D: DDRDDDDDD
L: BLLLLLLLL
B: BRRBBBBBB
State string: UULUUFUUFRRUBRRURRFFDFFUFFFDDRDDDDDDBLLLLLLLLBRRBBBBBB
Solved: False

=== After F R U R' U' F' ===
U: UULUUFUBL
R: UUURRRRRR
F: RUFFFFFFF
D: DDDDDDDDD
L: BLFLLLLLL
B: BRRBBBBBB
State string: UULUUFUBLUUURRRRRRRUFFFFFFFDDDDDDDDDBLFLLLLLLBRRBBBBBB
Solved: False

=== After T-Perm ===
U: UUUUUUUUU
R: RBBRRRRRR
F: BFFFFFFFF
D: DDDDDDDDD
L: FRRLLLLLL
B: LLLBBBBBB
State string: UUUUUUUUURBBRRRRRRBFFFFFFFFDDDDDDDDDFRRLLLLLLLLLBBBBBB
Solved: False


=== Advanced Display Options ===
The VCube class supports various display modes and options:
• show() method - prints visual representation
• display() method - returns visual represe

## Cubie Representation Integration: from_cubies and to_cubies

VCube seamlessly integrates with the cubie representation system, providing two key methods for converting between facelet and cubie representations:

In [5]:
print("=== Cubie Representation Integration ===")
print()
print("VCube provides seamless integration with cubie representation:")
print("• to_cubies property: Convert current cube state to cubie format")
print("• from_cubies() static method: Create VCube from cubie representation")
print("• Perfect round-trip conversion with no information loss")
print("• Integration with facelets conversion system for performance")
print()

def demonstrate_basic_cubie_conversion():
    """Demonstrate basic cubie conversion operations"""
    print("=== Basic Cubie Conversion ===")

    # Start with a scrambled cube
    cube = VCube()
    cube.rotate("R U R' F' R U R' U' R' F R2 U' R'")

    print(f"Original VCube state: {cube.state}")
    print(f"Is solved: {cube.is_solved}")

    # Convert to cubie representation
    print("\nConverting to cubie representation using .to_cubies:")
    cp, co, ep, eo, so = cube.to_cubies

    print(f"Corner Permutation (CP): {cp}")
    print(f"Corner Orientation (CO): {co}")
    print(f"Edge Permutation (EP): {ep}")
    print(f"Edge Orientation (EO): {eo}")
    print(f"Spatial Orientation (SO): {so}")

    # Create new VCube from cubies
    print("\nCreating new VCube using VCube.from_cubies():")
    reconstructed_cube = VCube.from_cubies(cp, co, ep, eo, so)

    print(f"Reconstructed state: {reconstructed_cube.state}")
    print(f"Perfect round-trip: {cube.state == reconstructed_cube.state}")
    print(f"Both solved status: {cube.is_solved} == {reconstructed_cube.is_solved}")

    return cube, reconstructed_cube

def analyze_cubie_components():
    """Analyze the components of cubie representation"""
    print("\n=== Understanding Cubie Components ===")

    # Create different cube states for analysis
    test_cases = [
        ("Solved cube", VCube()),
        ("After R move", VCube()),
        ("After Sexy move", VCube()),
        ("After T-Perm", VCube()),
    ]

    # Apply moves to create different states
    test_cases[1][1].rotate("R")
    test_cases[2][1].rotate("R U R' U'")
    test_cases[3][1].rotate("R U R' F' R U R' U' R' F R2 U' R'")

    print("Analyzing cubie components across different cube states:")
    print()
    print(f"{'State':<15} {'CP Sum':<8} {'CO Sum':<8} {'EP Sum':<8} {'EO Sum':<8} {'SO':<12}")
    print("-" * 70)

    for name, cube in test_cases:
        cp, co, ep, eo, so = cube.to_cubies

        cp_sum = sum(cp)
        co_sum = sum(co) % 3  # Corner orientation constraint
        ep_sum = sum(ep)
        eo_sum = sum(eo) % 2  # Edge orientation constraint

        print(f"{name:<15} {cp_sum:<8} {co_sum:<8} {ep_sum:<8} {eo_sum:<8} {str(so):<12}")

    print()
    print("MATHEMATICAL CONSTRAINTS:")
    print("• Corner orientations sum must be ≡ 0 (mod 3)")
    print("• Edge orientations sum must be ≡ 0 (mod 2)")
    print("• These constraints ensure cube solvability")

def demonstrate_cubie_manipulation():
    """Show how to manipulate cube states using cubie representation"""
    print("\n=== Cubie State Manipulation ===")

    # Start with solved cube
    original = VCube()
    cp, co, ep, eo, so = original.to_cubies

    print("Starting with solved cube:")
    print(f"Original state: {original.state}")
    print(f"CP: {cp}")
    print(f"CO: {co}")

    # Manipulate corner orientations (while maintaining constraint)
    print("\n1. MANIPULATING CORNER ORIENTATIONS:")
    modified_co = co.copy()
    # Twist corner 0 clockwise (add 1) and corner 1 counter-clockwise (add 2)
    # This maintains the sum ≡ 0 (mod 3) constraint: +1 +2 ≡ 0 (mod 3)
    modified_co[0] = (modified_co[0] + 1) % 3
    modified_co[1] = (modified_co[1] + 2) % 3

    twisted_cube = VCube.from_cubies(cp, modified_co, ep, eo, so)
    print(f"Modified CO: {modified_co}")
    print(f"New state: {twisted_cube.state}")
    print(f"Still valid: CO sum mod 3 = {sum(modified_co) % 3}")

    # Manipulate edge orientations (while maintaining constraint)
    print("\n2. MANIPULATING EDGE ORIENTATIONS:")
    modified_eo = eo.copy()
    # Flip two edges to maintain sum ≡ 0 (mod 2) constraint
    modified_eo[0] = 1 - modified_eo[0]  # Flip edge 0
    modified_eo[1] = 1 - modified_eo[1]  # Flip edge 1

    flipped_cube = VCube.from_cubies(cp, co, ep, modified_eo, so)
    print(f"Modified EO: {modified_eo[:6]}... (showing first 6)")
    print(f"New state: {flipped_cube.state}")
    print(f"Still valid: EO sum mod 2 = {sum(modified_eo) % 2}")

    # Manipulate permutations
    print("\n3. MANIPULATING PERMUTATIONS:")
    modified_cp = cp.copy()
    modified_ep = ep.copy()

    # Swap two corners and two edges (maintains parity)
    modified_cp[0], modified_cp[1] = modified_cp[1], modified_cp[0]
    modified_ep[0], modified_ep[1] = modified_ep[1], modified_ep[0]

    swapped_cube = VCube.from_cubies(modified_cp, co, modified_ep, eo, so)
    print(f"Swapped corners 0↔1 and edges 0↔1")
    print(f"New state: {swapped_cube.state}")
    print(f"Is solved: {swapped_cube.is_solved}")

    # Demonstrate spatial orientation
    print("\n4. MANIPULATING SPATIAL ORIENTATION:")
    rotated_so = [1, 2, 0, 4, 5, 3]  # Equivalent to 'y' rotation
    reoriented_cube = VCube.from_cubies(cp, co, ep, eo, rotated_so)

    print(f"Original SO: {so}")
    print(f"Rotated SO: {rotated_so}")
    print(f"Reoriented state: {reoriented_cube.state}")
    print(f"Still solved: {reoriented_cube.is_solved}")

def demonstrate_practical_applications():
    """Show practical applications of cubie integration"""
    print("\n=== Practical Applications ===")

    def create_custom_cube_state(description, cp=None, co=None, ep=None, eo=None, so=None):
        """Create a custom cube state from cubie components"""
        # Use solved state as defaults
        default_cp = [0, 1, 2, 3, 4, 5, 6, 7]
        default_co = [0, 0, 0, 0, 0, 0, 0, 0]
        default_ep = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
        default_eo = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        default_so = [0, 1, 2, 3, 4, 5]

        final_cp = cp if cp is not None else default_cp
        final_co = co if co is not None else default_co
        final_ep = ep if ep is not None else default_ep
        final_eo = eo if eo is not None else default_eo
        final_so = so if so is not None else default_so

        try:
            cube = VCube.from_cubies(final_cp, final_co, final_ep, final_eo, final_so)
            print(f"✓ {description}: Created successfully")
            return cube
        except Exception as e:
            print(f"✗ {description}: Failed - {e}")
            return None

    print("Creating specialized cube states:")

    # 1. Corner-twist only state
    corner_twist_co = [1, 2, 0, 0, 0, 0, 0, 0]  # Sum = 3 ≡ 0 (mod 3)
    corner_cube = create_custom_cube_state("Corner twist only", co=corner_twist_co)

    # 2. Edge-flip only state
    edge_flip_eo = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # Sum = 2 ≡ 0 (mod 2)
    edge_cube = create_custom_cube_state("Edge flip only", eo=edge_flip_eo)

    # 3. Permutation-only state (3-cycle of corners)
    perm_cp = [1, 2, 0, 3, 4, 5, 6, 7]  # 3-cycle: 0→1→2→0
    perm_ep = [1, 2, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11]  # 3-cycle: 0→1→2→0
    perm_cube = create_custom_cube_state("Permutation only", cp=perm_cp, ep=perm_ep)

    # 4. Different orientation
    oriented_so = [3, 4, 5, 0, 1, 2]  # x2 rotation
    oriented_cube = create_custom_cube_state("Different orientation", so=oriented_so)

    # Analyze the created cubes
    valid_cubes = [cube for cube in [corner_cube, edge_cube, perm_cube, oriented_cube] if cube]

    if valid_cubes:
        print(f"\nAnalyzing {len(valid_cubes)} created cube states:")
        print(f"{'Type':<20} {'Solved':<8} {'State Preview':<25}")
        print("-" * 60)

        cube_types = ["Corner twist", "Edge flip", "Permutation", "Reoriented"]
        for i, cube in enumerate(valid_cubes):
            if i < len(cube_types) and cube:
                state_preview = cube.state[:20] + "..."
                print(f"{cube_types[i]:<20} {str(cube.is_solved):<8} {state_preview:<25}")

def benchmark_cubie_performance():
    """Benchmark cubie conversion performance"""
    print("\n=== Performance Benchmarking ===")
    import time

    # Create test cube states
    test_cubes = []
    base_cube = VCube()

    # Generate different scrambled states
    scrambles = [
        "R", "R U R'", "R U R' U'",
        "R U R' F' R U R' U' R' F R2 U' R'",
        "M2 U M2 U2 M2 U M2",
    ]

    for scramble in scrambles:
        cube = VCube()
        cube.rotate(scramble)
        test_cubes.append(cube)

    print(f"Testing with {len(test_cubes)} different cube states:")

    # Benchmark to_cubies performance
    start_time = time.time()
    for _ in range(1000):
        for cube in test_cubes:
            cubies = cube.to_cubies
    to_cubies_time = time.time() - start_time

    # Benchmark from_cubies performance
    cubies_list = [cube.to_cubies for cube in test_cubes]
    start_time = time.time()
    for _ in range(1000):
        for cp, co, ep, eo, so in cubies_list:
            cube = VCube.from_cubies(cp, co, ep, eo, so)
    from_cubies_time = time.time() - start_time

    # Benchmark round-trip performance
    start_time = time.time()
    for _ in range(1000):
        for cube in test_cubes:
            cp, co, ep, eo, so = cube.to_cubies
            reconstructed = VCube.from_cubies(cp, co, ep, eo, so)
    round_trip_time = time.time() - start_time

    total_operations = 1000 * len(test_cubes)

    print(f"\nPerformance Results ({total_operations:,} operations each):")
    print(f"• to_cubies:   {to_cubies_time:.4f}s ({total_operations/to_cubies_time:,.0f} ops/sec)")
    print(f"• from_cubies: {from_cubies_time:.4f}s ({total_operations/from_cubies_time:,.0f} ops/sec)")
    print(f"• Round-trip:  {round_trip_time:.4f}s ({total_operations/round_trip_time:,.0f} ops/sec)")

    # Test accuracy
    print(f"\nAccuracy verification:")
    accuracy_errors = 0
    for cube in test_cubes:
        cp, co, ep, eo, so = cube.to_cubies
        reconstructed = VCube.from_cubies(cp, co, ep, eo, so)
        if cube.state != reconstructed.state:
            accuracy_errors += 1

    print(f"• Round-trip accuracy: {len(test_cubes) - accuracy_errors}/{len(test_cubes)} perfect")
    if accuracy_errors == 0:
        print("• ✓ All conversions were perfectly accurate")

# Run all demonstrations
demonstrate_basic_cubie_conversion()
analyze_cubie_components()
demonstrate_cubie_manipulation()
demonstrate_practical_applications()
benchmark_cubie_performance()

print("\n" + "="*70)
print("CUBIE INTEGRATION SUMMARY:")
print("• .to_cubies: Convert VCube state to mathematical cubie representation")
print("• .from_cubies(): Create VCube from cubie components")
print("• Perfect accuracy: No information loss in round-trip conversions")
print("• High performance: Thousands of conversions per second")
print("• Mathematical constraints: Automatic validation of cube solvability")
print("• Flexible manipulation: Create custom cube states programmatically")
print("• Seamless integration: Works with existing VCube methods and properties")
print("="*70)

=== Cubie Representation Integration ===

VCube provides seamless integration with cubie representation:
• to_cubies property: Convert current cube state to cubie format
• from_cubies() static method: Create VCube from cubie representation
• Perfect round-trip conversion with no information loss
• Integration with facelets conversion system for performance

=== Basic Cubie Conversion ===
Original VCube state: UUUUUUUUURBBRRRRRRBFFFFFFFFDDDDDDDDDFRRLLLLLLLLLBBBBBB
Is solved: False

Converting to cubie representation using .to_cubies:
Corner Permutation (CP): [0, 3, 1, 2, 4, 5, 6, 7]
Corner Orientation (CO): [0, 0, 0, 0, 0, 0, 0, 0]
Edge Permutation (EP): [3, 1, 0, 2, 4, 5, 6, 7, 8, 9, 10, 11]
Edge Orientation (EO): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Spatial Orientation (SO): [0, 1, 2, 3, 4, 5]

Creating new VCube using VCube.from_cubies():
Reconstructed state: UUUUUUUUURBBRRRRRRBFFFFFFFFDDDDDDDDDFRRLLLLLLLLLBBBBBB
Perfect round-trip: True
Both solved status: False == False

=== Unders

## Cube Orientation and Reorientation

One of the powerful features of VCube is its ability to handle cube orientation and create reoriented copies:

In [6]:
print("=== Cube Orientation and Reorientation ===")

def demonstrate_orientation(cube, name):
    """Demonstrate orientation features of a cube"""
    print(f"\n=== {name} ===")
    print(f"State: {cube.state}")
    print(f"Orientation: {cube.orientation}")
    print(f"Is solved: {cube.is_solved}")
    
    # Show face centers
    centers = cube.get_face_center_indexes()
    face_centers = {face: center for face, center in zip(FACE_ORDER, centers)}
    print(f"Face centers: {face_centers}")
    
    return cube

# Create a base cube with some moves
base_cube = VCube()
base_cube.rotate("R U R' F' R U R'")
demonstrate_orientation(base_cube, "Base Cube")

# Apply cube rotations to change orientation
rotated_cube = base_cube.copy()
rotated_cube.rotate("y")  # Rotate cube around Y-axis
demonstrate_orientation(rotated_cube, "After y rotation")

rotated_cube2 = base_cube.copy()
rotated_cube2.rotate("x z")  # Multiple rotations
demonstrate_orientation(rotated_cube2, "After x z rotations")

print("\n=== Creating Oriented Copies ===")

# Create oriented copies
try:
    # Create copy oriented with U top and F front
    uf_oriented = base_cube.oriented_copy("UF")
    demonstrate_orientation(uf_oriented, "Oriented to UF")
    
    # Create copy oriented with D top and R front  
    dr_oriented = base_cube.oriented_copy("DR")
    demonstrate_orientation(dr_oriented, "Oriented to DR")
    
    # Create copy oriented with R top and B front
    rb_oriented = base_cube.oriented_copy("RB")
    demonstrate_orientation(rb_oriented, "Oriented to RB")
    
except Exception as e:
    print(f"Orientation error: {e}")
    print("This might happen with invalid orientations or corrupted states")

print("\n=== Orientation Moves Calculation ===")
print("VCube can calculate what moves are needed to achieve orientations:")

# Test orientation move calculation
test_orientations = ["UF", "UR", "UB", "UL", "DF", "RF", "LB"]

for orientation in test_orientations:
    try:
        moves = base_cube.compute_orientation_moves(orientation)
        print(f"To achieve {orientation} orientation: '{moves}'")
    except Exception as e:
        print(f"Cannot compute moves for {orientation}: {e}")

print("\n=== Cube Equality with Orientation ===")

# Create two cubes with same pattern but different orientations
cube1 = VCube()
cube1.rotate("R U R' U'")

cube2 = VCube()
cube2.rotate("R U R' U'")  # Same pattern
cube2.rotate("y")  # Rotate 

print(f"Cube 1 state: {cube1.state}")
print(f"Cube 2 state: {cube2.state}")
print(f"Cube 1 orientation: {cube1.orientation}")
print(f"Cube 2 orientation: {cube2.orientation}")

# Test equality
strict_equal = cube1.is_equal(cube2, strict=True)
flexible_equal = cube1.is_equal(cube2, strict=False)

print(f"\nStrict equality: {strict_equal}")
print(f"Flexible equality (accounting for orientation): {flexible_equal}")

print("\n" + "="*60)
print("ORIENTATION CONCEPTS:")
print("• Orientation is determined by top and front face centers")
print("• oriented_copy() creates a new cube with specified orientation")
print("• compute_orientation_moves() calculates required rotations")
print("• is_equal() can compare cubes with/without considering orientation")
print("• Useful for algorithm analysis across different cube orientations")
print("="*60)

=== Cube Orientation and Reorientation ===

=== Base Cube ===
State: BUUBUUFFFLRRLRRURRDDURFURFFDLRDDDDDDUULLLFLLFBFLBBBBBB
Orientation: UF
Is solved: False
Face centers: {'U': 'U', 'R': 'R', 'F': 'F', 'D': 'D', 'L': 'L', 'B': 'B'}

=== After y rotation ===
State: FBBFUUFUUBFLBBBBBBLRRLRRURRRDDLDDDDDDDURFURFFUULLLFLLF
Orientation: UR
Is solved: False
Face centers: {'U': 'U', 'R': 'B', 'F': 'R', 'D': 'D', 'L': 'F', 'B': 'L'}

=== After x z rotations ===
State: UULLLFLLFRRDFFDFUUDDDDDLDDRRRURRLRRLLBBFBBBBBFBBFUUFUU
Orientation: LD
Is solved: False
Face centers: {'U': 'L', 'R': 'F', 'F': 'D', 'D': 'R', 'L': 'B', 'B': 'U'}

=== Creating Oriented Copies ===

=== Oriented to UF ===
State: BUUBUUFFFLRRLRRURRDDURFURFFDLRDDDDDDUULLLFLLFBFLBBBBBB
Orientation: UF
Is solved: False
Face centers: {'U': 'U', 'R': 'R', 'F': 'F', 'D': 'D', 'L': 'L', 'B': 'B'}

=== Oriented to DR ===
State: DDDDDLDDRFFRUFRUDDRRURRLRRLUUFUUFBBFBBBBBBLFBFLLFLLLUU
Orientation: DR
Is solved: False
Face centers: {'U': 'D', '

## Cube State Integrity and Validation

VCube includes comprehensive integrity checking to ensure cube states are valid and solvable:

In [7]:
print("=== Cube State Integrity and Validation ===")

def test_cube_validity(state_string, description):
    """Test if a cube state is valid"""
    print(f"\nTesting: {description}")
    print(f"State: {state_string}")
    
    try:
        # Try to create cube with integrity checking enabled (default)
        cube = VCube(state_string, check=True)
        print("✓ Valid cube state")
        print(f"  Is solved: {cube.is_solved}")
        print(f"  Orientation: {cube.orientation}")
        return cube
    except Exception as e:
        print(f"✗ Invalid cube state: {e}")
        return None

# Test various cube states
print("1. VALID STATES")
print("-" * 30)

valid_states = [
    (INITIAL_STATE, "Standard solved cube"),
    ("UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB", "Solved state (explicit)"),
]

# Create a valid scrambled state by applying moves to solved cube
scrambled_cube = VCube()
scrambled_cube.rotate("R U R' U' R U R' F' R U R' U' R' F R2 U' R'")
valid_states.append((scrambled_cube.state, "Scrambled with T-Perm"))

valid_cubes = []
for state, description in valid_states:
    cube = test_cube_validity(state, description)
    if cube:
        valid_cubes.append(cube)

print("\n2. INVALID STATES")
print("-" * 30)

invalid_states = [
    ("INVALID_STATE_TOO_SHORT", "State too short"),
    ("U" * 100, "State too long"),
    ("X" * 54, "Invalid color characters"),
    ("UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU", "All same color"),
    ("UUUUUUUUURRRRRRRRR123456789DDDDDDDDDLLLLLLLLLBBBBBBBBB", "Invalid characters in middle"),
]

# Test invalid states
for state, description in invalid_states:
    test_cube_validity(state, description)

print("\n3. EDGE CASE TESTING")
print("-" * 30)

# Create cube without integrity checking
print("Creating cube without integrity checking:")
try:
    unchecked_cube = VCube("X" * 54, check=False)
    print("✓ Unchecked cube created successfully")
    print(f"  State: {unchecked_cube.state[:20]}...")
    print(f"  Is solved: {unchecked_cube.is_solved}")
    
    # Try to check integrity manually
    print("\nManually checking integrity:")
    try:
        unchecked_cube.check_integrity()
        print("✓ Manual integrity check passed")
    except Exception as e:
        print(f"✗ Manual integrity check failed: {e}")
        
except Exception as e:
    print(f"✗ Could not create unchecked cube: {e}")

print("\n4. CUBE INTEGRITY CONCEPTS")
print("-" * 40)
print("VCube integrity checking validates:")
print("• State string length (must be 54 characters)")
print("• Valid color characters (U, R, F, D, L, B)")
print("• Correct number of each color (9 of each)")
print("• Valid corner permutation and orientation")
print("• Valid edge permutation and orientation")
print("• Cube solvability (mathematical constraints)")

print("\n5. WORKING WITH CUBIE REPRESENTATION")
print("-" * 45)

# Demonstrate cubie conversion
if valid_cubes:
    demo_cube = valid_cubes[0]
    print(f"Demo cube state: {demo_cube.state}")
    
    try:
        # Convert to cubie representation
        cp, co, ep, eo, so = demo_cube.to_cubies
        print(f"Corner permutation: {cp}")
        print(f"Corner orientation: {co}")
        print(f"Edge permutation: {ep}")
        print(f"Edge orientation: {eo}")
        print(f"Center permutation: {so}")
        
        # Create cube from cubie representation
        reconstructed_cube = VCube.from_cubies(cp, co, ep, eo, so)
        print(f"\nReconstructed state: {reconstructed_cube.state}")
        print(f"States match: {demo_cube.state == reconstructed_cube.state}")
        
    except Exception as e:
        print(f"Error with cubie representation: {e}")

print("\n" + "="*60)
print("INTEGRITY CHECKING BENEFITS:")
print("• Prevents invalid cube states from being processed")
print("• Ensures cube states are mathematically solvable")
print("• Catches data corruption and input errors early")
print("• Provides clear error messages for debugging")
print("• Can be disabled for performance in trusted scenarios")
print("="*60)

=== Cube State Integrity and Validation ===
1. VALID STATES
------------------------------

Testing: Standard solved cube
State: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
✓ Valid cube state
  Is solved: True
  Orientation: UF

Testing: Solved state (explicit)
State: UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB
✓ Valid cube state
  Is solved: True
  Orientation: UF

Testing: Scrambled with T-Perm
State: UUUFUULUFRRRBRRURRBFDFFUFFFDDRDDDDDDFRULLLLLLBLLBBBBBB
✓ Valid cube state
  Is solved: False
  Orientation: UF

2. INVALID STATES
------------------------------

Testing: State too short
State: INVALID_STATE_TOO_SHORT
✗ Invalid cube state: State string must be 54 characters long

Testing: State too long
State: UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU
✗ Invalid cube state: State string must be 54 characters long

Testing: Invalid color characters
State: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

## Practical Cube Simulation Examples

Let's explore practical applications of VCube for speedcubing simulation and analysis:

In [8]:
print("=== Practical Cube Simulation Examples ===")

def simulate_solve_attempt(scramble, solution, description):
    """Simulate a complete solve attempt"""
    print(f"\n=== {description} ===")
    
    # Start with solved cube
    cube = VCube()
    print(f"Starting state: Solved = {cube.is_solved}")
    
    # Apply scramble
    cube.rotate(scramble)
    print(f"Scramble: {scramble}")
    print(f"After scramble: Solved = {cube.is_solved}")
    
    # Apply solution
    cube.rotate(solution)
    print(f"Solution: {solution}")
    print(f"After solution: Solved = {cube.is_solved}")
    
    # Show move count
    total_moves = len(cube.history)
    scramble_moves = len(scramble.split())
    solution_moves = len(solution.split())
    print(f"Total moves in history: {total_moves} (Scramble: {scramble_moves}, Solution: {solution_moves})")
    
    return cube

# Example 1: Simple 2x2 pattern solve
simulate_solve_attempt(
    "R U R' U'",
    "U R U' R'",
    "Simple Pattern Solve"
)

# Example 2: T-Perm scenario
simulate_solve_attempt(
    "R U R' F' R U R' U' R' F R2 U' R'",
    "R U R' F' R U R' U' R' F R2 U' R'",
    "T-Perm Self-Solve"
)

print("\n=== Algorithm Testing Framework ===")

def test_algorithm_properties(algorithm_str, name):
    """Test various properties of an algorithm"""
    print(f"\nTesting: {name}")
    print(f"Algorithm: {algorithm_str}")
    
    cube = VCube()
    original_state = cube.state
    
    # Apply algorithm multiple times to find cycle
    applications = 0
    max_applications = 20
    
    while applications < max_applications:
        cube.rotate(algorithm_str)
        applications += 1
        
        if cube.state == original_state:
            print(f"  Cycle found after {applications} applications")
            break
    else:
        print(f"  No cycle found within {max_applications} applications")
    
    # Test effect on cube
    test_cube = VCube()
    test_cube.rotate(algorithm_str)
    
    changed_facelets = sum(1 for a, b in zip(original_state, test_cube.state) if a != b)
    print(f"  Affects {changed_facelets}/54 facelets")
    print(f"  Leaves cube solved: {test_cube.is_solved}")
    
    return applications if applications < max_applications else None

# Test common algorithms
algorithms_to_test = [
    ("R", "Single R turn"),
    ("R2", "Double R turn"),
    ("R2 R2", "R turn repeated 4 times"),
    ("R U R' U'", "Sexy move"),
    ("R U R' U R U2 R'", "Sune"),
    ("R U2 R' U' R U' R'", "Antisune"),
    ("M2 U M2 U2 M2 U M2", "H-Perm"),
]

cycle_results = []
for alg, name in algorithms_to_test:
    try:
        cycle_count = test_algorithm_properties(alg, name)
        cycle_results.append((name, alg, cycle_count))
    except Exception as e:
        print(f"  Error testing {name}: {e}")
        cycle_results.append((name, alg, None))

print("\n=== Cycle Summary ===")
print(f"{'Algorithm':<15} {'Cycle Count':<12} {'Classification':<15}")
print("-" * 50)
for name, alg, cycle_count in cycle_results:
    if cycle_count is None:
        classification = "Error/High order"
        cycle_str = "N/A"
    elif cycle_count == 1:
        classification = "Identity"
        cycle_str = "1"
    elif cycle_count == 2:
        classification = "Self-inverse"
        cycle_str = "2"
    else:
        classification = "Complex"
        cycle_str = str(cycle_count)
    
    print(f"{name[:14]:<15} {cycle_str:<12} {classification:<15}")

print("\n=== State Comparison and Analysis ===")

def compare_cube_states(cube1, cube2, name1="Cube 1", name2="Cube 2"):
    """Compare two cube states in detail"""
    print(f"\nComparing {name1} vs {name2}:")
    
    # Basic comparison
    exact_match = cube1.state == cube2.state
    orientation_match = cube1.is_equal(cube2, strict=False)
    
    print(f"  Exact state match: {exact_match}")
    print(f"  Orientation-flexible match: {orientation_match}")
    
    if not exact_match:
        # Count differences
        differences = sum(1 for a, b in zip(cube1.state, cube2.state) if a != b)
        print(f"  Different facelets: {differences}/54")
        
        # Show first few differences
        diff_positions = [(i, a, b) for i, (a, b) in enumerate(zip(cube1.state, cube2.state)) if a != b]
        if diff_positions:
            print(f"  First 5 differences: {diff_positions[:5]}")
    
    print(f"  {name1} solved: {cube1.is_solved}, {name2} solved: {cube2.is_solved}")
    print(f"  {name1} orientation: {cube1.orientation}, {name2} orientation: {cube2.orientation}")

# Create some cubes for comparison
cube_a = VCube()
cube_a.rotate("R U R' U'")

cube_b = VCube()
cube_b.rotate("U R U' R'")

cube_c = VCube()
cube_c.rotate("y R U R' U'")

compare_cube_states(cube_a, cube_b, "After R U R' U'", "After U R U' R'")
compare_cube_states(cube_a, cube_c, "After R U R' U'", "After y R U R' U'")
compare_cube_states(cube_b, cube_c, "After U R U' R'", "After y R U R' U'")

print("\n=== Practical Applications Summary ===")
print("✓ Algorithm cycle analysis and classification")
print("✓ Solve simulation and verification")
print("✓ State comparison and difference analysis")
print("✓ Move history tracking for analysis")
print("✓ Algorithm testing and validation")
print("✓ Pattern recognition and matching")

=== Practical Cube Simulation Examples ===

=== Simple Pattern Solve ===
Starting state: Solved = True
Scramble: R U R' U'
After scramble: Solved = False
Solution: U R U' R'
After solution: Solved = True
Total moves in history: 8 (Scramble: 4, Solution: 4)

=== T-Perm Self-Solve ===
Starting state: Solved = True
Scramble: R U R' F' R U R' U' R' F R2 U' R'
After scramble: Solved = False
Solution: R U R' F' R U R' U' R' F R2 U' R'
After solution: Solved = False
Total moves in history: 26 (Scramble: 13, Solution: 13)

=== Algorithm Testing Framework ===

Testing: Single R turn
Algorithm: R
  Cycle found after 4 applications
  Affects 12/54 facelets
  Leaves cube solved: False

Testing: Double R turn
Algorithm: R2
  Cycle found after 2 applications
  Affects 12/54 facelets
  Leaves cube solved: False

Testing: R turn repeated 4 times
Algorithm: R2 R2
  Cycle found after 1 applications
  Affects 0/54 facelets
  Leaves cube solved: True

Testing: Sexy move
Algorithm: R U R' U'
  Cycle found 

## Advanced VCube Features and Integration

Let's explore advanced features and how VCube integrates with other components of the cubing_algs library:

In [9]:
print("=== Advanced VCube Features and Integration ===")

def demonstrate_copy_features():
    """Demonstrate different copy options"""
    print("\n1. COPY FUNCTIONALITY")
    print("-" * 30)
    
    # Create original cube with history
    original = VCube()
    original.rotate("R U R' F' R U R'")
    print(f"Original cube history: {original.history}")
    print(f"Original state: {original.state[:20]}...")
    
    # Basic copy (no history)
    basic_copy = original.copy()
    print(f"Basic copy history: {basic_copy.history}")
    print(f"Basic copy state matches: {basic_copy.state == original.state}")
    
    # Full copy (with history)
    full_copy = original.copy(full=True)
    print(f"Full copy history: {full_copy.history}")
    print(f"Full copy history matches: {full_copy.history == original.history}")
    
    # Modify copies to show independence
    basic_copy.rotate("U")
    full_copy.rotate("D")
    
    print(f"\nAfter modifications:")
    print(f"Original history: {len(original.history)} moves")
    print(f"Basic copy history: {len(basic_copy.history)} moves")
    print(f"Full copy history: {len(full_copy.history)} moves")
    
    print(f"States still independent: {len(set([original.state, basic_copy.state, full_copy.state])) == 3}")

def demonstrate_algorithm_integration():
    """Show how VCube integrates with Algorithm class"""
    print("\n2. ALGORITHM INTEGRATION")
    print("-" * 35)
    
    # Create algorithm object
    t_perm = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R'")
    print(f"Algorithm: {t_perm}")
    print(f"Algorithm length: {len(t_perm)}")
    print(f"Algorithm cycles: {t_perm.cycles}")
    
    # Use algorithm's show method (creates VCube internally)
    print("\nUsing Algorithm.show() method:")
    try:
        result_cube = t_perm.show()
        print(f"Result cube type: {type(result_cube).__name__}")
        print(f"Result cube solved: {result_cube.is_solved}")
        print(f"Result cube state: {result_cube.state[:20]}...")
    except Exception as e:
        print(f"Algorithm show error: {e}")
    
    # Manual VCube with Algorithm
    manual_cube = VCube()
    print(f"\nBefore algorithm: {manual_cube.is_solved}")
    manual_cube.rotate(t_perm)
    print(f"After algorithm: {manual_cube.is_solved}")
    print(f"History length: {len(manual_cube.history)}")

def demonstrate_state_manipulation():
    """Show advanced state manipulation techniques"""
    print("\n3. STATE MANIPULATION")
    print("-" * 30)
    
    # Create custom states
    print("Creating custom cube states:")
    
    # Solved cube
    solved = VCube()
    print(f"Solved: {solved.orientation}")
    
    # Create scrambled state
    scrambled = VCube()
    scrambled.rotate("R U2 R' U' R U R' U' R' F R F' U R U' R'")
    print(f"Scrambled: {scrambled.orientation}")
    
    # Extract and manipulate individual faces
    print("\nFace manipulation:")
    for face in FACE_ORDER:
        face_state = scrambled.get_face(face)
        unique_colors = len(set(face_state))
        print(f"  {face} face has {unique_colors} unique colors: {face_state}")
    
    # Orientation-based operations
    print("\nOrientation operations:")
    try:
        # Create copies with different orientations
        orientations = ["UF", "UR", "UB", "UL"]
        oriented_cubes = []
        
        for orient in orientations:
            try:
                oriented = scrambled.oriented_copy(orient)
                oriented_cubes.append((orient, oriented))
                print(f"  {orient} orientation: {oriented.state[:10]}...")
            except Exception as e:
                print(f"  {orient} orientation failed: {e}")
        
        # Check if they're equivalent under flexible equality
        if len(oriented_cubes) >= 2:
            cube1 = oriented_cubes[0][1]
            cube2 = oriented_cubes[1][1]
            flexible_equal = cube1.is_equal(cube2, strict=False)
            print(f"  Different orientations are equivalent: {flexible_equal}")
    
    except Exception as e:
        print(f"  Orientation operations error: {e}")

def demonstrate_performance_considerations():
    """Show performance considerations for VCube usage"""
    print("\n4. PERFORMANCE CONSIDERATIONS")
    print("-" * 40)
    
    import time
    
    # Test cube creation performance
    print("Cube creation performance:")
    
    # Time cube creation with integrity checking
    start_time = time.time()
    for _ in range(1000):
        cube = VCube()
    checked_time = time.time() - start_time
    
    # Time cube creation without integrity checking
    start_time = time.time()
    for _ in range(1000):
        cube = VCube(check=False)
    unchecked_time = time.time() - start_time
    
    print(f"  1000 cubes with checking: {checked_time:.4f}s")
    print(f"  1000 cubes without checking: {unchecked_time:.4f}s")
    print(f"  Speedup: {checked_time/unchecked_time:.1f}x faster without checking")
    
    # Test move application performance
    print("\nMove application performance:")
    
    cube = VCube()
    test_moves = "R U R' U' F R F' U R U' R'"
    
    # With history
    start_time = time.time()
    for _ in range(1000):
        test_cube = VCube()
        test_cube.rotate(test_moves, history=True)
    with_history_time = time.time() - start_time
    
    # Without history
    start_time = time.time()
    for _ in range(1000):
        test_cube = VCube()
        test_cube.rotate(test_moves, history=False)
    without_history_time = time.time() - start_time
    
    print(f"  1000 sequences with history: {with_history_time:.4f}s")
    print(f"  1000 sequences without history: {without_history_time:.4f}s")
    print(f"  Speedup: {with_history_time/without_history_time:.1f}x faster without history")
    
    # Copy performance
    print("\nCopy performance:")
    source_cube = VCube()
    source_cube.rotate("R U R' U' F R F' U R U' R' D L D' B U B'")
    
    # Basic copy
    start_time = time.time()
    for _ in range(1000):
        copy = source_cube.copy()
    basic_copy_time = time.time() - start_time
    
    # Full copy
    start_time = time.time()
    for _ in range(1000):
        copy = source_cube.copy(full=True)
    full_copy_time = time.time() - start_time
    
    print(f"  1000 basic copies: {basic_copy_time:.4f}s")
    print(f"  1000 full copies: {full_copy_time:.4f}s")
    print(f"  Full copy overhead: {full_copy_time/basic_copy_time:.1f}x slower")

# Run all demonstrations
demonstrate_copy_features()
demonstrate_algorithm_integration()
demonstrate_state_manipulation()
demonstrate_performance_considerations()

print("\n=== Integration Benefits ===")
print("✓ Seamless Algorithm object support")
print("✓ Flexible copy options for different use cases")
print("✓ Performance optimization options (check=False, history=False)")
print("✓ Advanced state manipulation and analysis")
print("✓ Orientation handling for algorithm analysis")
print("✓ Memory-efficient operations for large-scale processing")

=== Advanced VCube Features and Integration ===

1. COPY FUNCTIONALITY
------------------------------
Original cube history: ['R', 'U', "R'", "F'", 'R', 'U', "R'"]
Original state: BUUBUUFFFLRRLRRURRDD...
Basic copy history: []
Basic copy state matches: True
Full copy history: ['R', 'U', "R'", "F'", 'R', 'U', "R'"]
Full copy history matches: True

After modifications:
Original history: 7 moves
Basic copy history: 1 moves
Full copy history: 8 moves
States still independent: True

2. ALGORITHM INTEGRATION
-----------------------------------
Algorithm: R U R' F' R U R' U' R' F R2 U' R'
Algorithm length: 13
Algorithm cycles: 3

Using Algorithm.show() method:
          U  U  U 
          U  U  U 
          U  U  U 
 F  R  R  B  F  F  R  B  B  L  L  L 
 L  L  L  F  F  F  R  R  R  B  B  B 
 L  L  L  F  F  F  R  R  R  B  B  B 
          D  D  D 
          D  D  D 
          D  D  D 
Result cube type: VCube
Result cube solved: False
Result cube state: UUUUUUUUURBBRRRRRRBF...

Before algorithm: T

## Error Handling and Edge Cases

Understanding how VCube handles errors and edge cases is crucial for robust applications:

In [10]:
print("=== Error Handling and Edge Cases ===")

def test_move_errors():
    """Test various move-related error conditions"""
    print("\n1. MOVE APPLICATION ERRORS")
    print("-" * 35)
    
    cube = VCube()
    
    # Test invalid move strings
    invalid_moves = [
        "Z",       # Invalid move letter
        "R3",      # Invalid modifier
        "R'2",     # Invalid modifier combination
        "XYZ",     # Multiple invalid letters
        "R U Z",   # Mixed valid/invalid
        "",        # Empty string
        "   ",     # Whitespace only
    ]
    
    for invalid_move in invalid_moves:
        try:
            test_cube = cube.copy()
            test_cube.rotate(invalid_move)
            print(f"  '{invalid_move}' -> Unexpectedly succeeded")
        except Exception as e:
            print(f"  '{invalid_move}' -> {type(e).__name__}: {str(e)[:50]}...")

def test_state_errors():
    """Test cube state related errors"""
    print("\n2. STATE INITIALIZATION ERRORS")
    print("-" * 40)
    
    # Test invalid initial states
    invalid_states = [
        ("short", "Too short"),
        ("U" * 100, "Too long"),
        ("UUUUUUUUURRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBB123", "Invalid characters"),
        (None, "None value"),
        (123, "Non-string type"),
        ("UUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU", "All same color"),
    ]
    
    for state, description in invalid_states:
        try:
            cube = VCube(state, check=True)
            print(f"  {description} -> Unexpectedly succeeded")
        except Exception as e:
            print(f"  {description} -> {type(e).__name__}: {str(e)[:50]}...")

def test_orientation_errors():
    """Test orientation-related error conditions"""
    print("\n3. ORIENTATION ERRORS")
    print("-" * 25)
    
    cube = VCube()
    
    # Test invalid orientations
    invalid_orientations = [
        "UU",    # Same face twice
        "UD",    # Opposite faces
        "XY",    # Invalid face letters
        "UFB",   # Too long
        "",      # Empty
        "uf",    # Wrong case
    ]
    
    for orientation in invalid_orientations:
        try:
            oriented_cube = cube.oriented_copy(orientation)
            print(f"  '{orientation}' -> Unexpectedly succeeded")
        except Exception as e:
            print(f"  '{orientation}' -> {type(e).__name__}: {str(e)[:50]}...")

def test_edge_cases():
    """Test various edge cases"""
    print("\n4. EDGE CASES")
    print("-" * 15)
    
    # Empty moves
    print("Empty move sequences:")
    cube = VCube()
    original_state = cube.state
    
    try:
        cube.rotate("")
        print(f"  Empty string: State unchanged = {cube.state == original_state}")
    except Exception as e:
        print(f"  Empty string: {type(e).__name__}: {e}")
    
    try:
        cube.rotate("   ")
        print(f"  Whitespace only: State unchanged = {cube.state == original_state}")
    except Exception as e:
        print(f"  Whitespace only: {type(e).__name__}: {e}")
    
    # Large number of moves
    print("\nLarge move sequences:")
    try:
        large_sequence = "R U R' U' " * 100  # 400 moves
        test_cube = VCube()
        test_cube.rotate(large_sequence.strip())
        print(f"  400 moves: Success, history length = {len(test_cube.history)}")
    except Exception as e:
        print(f"  400 moves: {type(e).__name__}: {e}")
    
    # State consistency
    print("\nState consistency:")
    cube1 = VCube()
    cube2 = VCube()
    
    # Apply same moves to both cubes
    moves = "R U R' F' R U R' U' R' F R2 U' R'"
    cube1.rotate(moves)
    
    # Apply moves one by one to second cube
    for move in moves.split():
        cube2.rotate_move(move)
    
    print(f"  Batch vs individual moves: States match = {cube1.state == cube2.state}")
    print(f"  History lengths: {len(cube1.history)} vs {len(cube2.history)}")

def demonstrate_safe_operations():
    """Show safe ways to work with VCube"""
    print("\n5. SAFE OPERATION PATTERNS")
    print("-" * 35)
    
    def safe_cube_creation(state=None, description="cube"):
        """Safely create a cube with error handling"""
        try:
            if state is None:
                cube = VCube()
            else:
                # Try with checking first
                cube = VCube(state, check=True)
            print(f"  ✓ {description} created successfully")
            return cube
        except Exception as e:
            print(f"  ✗ {description} creation failed: {e}")
            return None
    
    def safe_move_application(cube, moves, description="moves"):
        """Safely apply moves with error handling"""
        if cube is None:
            return None
        
        try:
            original_state = cube.state
            cube.rotate(moves)
            print(f"  ✓ {description} applied successfully")
            return cube
        except Exception as e:
            print(f"  ✗ {description} application failed: {e}")
            # Could restore state here if needed
            return cube
    
    # Demonstrate safe patterns
    print("Safe cube operations:")
    
    # Create cube safely
    cube = safe_cube_creation()
    
    # Apply moves safely
    cube = safe_move_application(cube, "R U R' U'", "sexy move")
    cube = safe_move_application(cube, "F R F'", "additional moves")
    
    # Try unsafe operation
    cube = safe_move_application(cube, "R U Z R'", "invalid moves")
    
    # Check final state
    if cube:
        print(f"  Final state: {'Solved' if cube.is_solved else 'Scrambled'}")
        print(f"  History length: {len(cube.history)}")

# Run all error handling tests
test_move_errors()
test_state_errors()
test_orientation_errors()
test_edge_cases()
demonstrate_safe_operations()

print("\n" + "="*60)
print("ERROR HANDLING BEST PRACTICES:")
print("• Always use try-except blocks when creating cubes from external data")
print("• Validate move strings before applying to prevent invalid moves")
print("• Use check=False for performance with trusted cube states")
print("• Handle orientation errors gracefully in user interfaces")
print("• Implement state backup/restore for critical operations")
print("• Use safe operation patterns for production code")
print("• Test edge cases thoroughly (empty strings, large sequences)")
print("="*60)

=== Error Handling and Edge Cases ===

1. MOVE APPLICATION ERRORS
-----------------------------------
  'Z' -> InvalidMoveError: Invalid move face: 'Z'...
  'R3' -> InvalidMoveError: Invalid move modifier: '3'...
  'R'2' -> Unexpectedly succeeded
  'XYZ' -> InvalidMoveError: Invalid move modifier: 'Y'...
  'R U Z' -> InvalidMoveError: Invalid move face: 'Z'...
  '' -> Unexpectedly succeeded
  '   ' -> InvalidMoveError: Invalid move face: ' '...

2. STATE INITIALIZATION ERRORS
----------------------------------------
  Too short -> InvalidCubeStateError: State string must be 54 characters long...
  Too long -> InvalidCubeStateError: State string must be 54 characters long...
  Invalid characters -> InvalidCubeStateError: State string can only contains U R F D L B charact...
  None value -> Unexpectedly succeeded
  Non-string type -> TypeError: object of type 'int' has no len()...
  All same color -> InvalidCubeStateError: State string must have 9 of each color...

3. ORIENTATION ERRORS


## Summary and Key Takeaways

The `VCube` class is the core simulation engine of the cubing_algs library, providing comprehensive virtual cube functionality:

In [11]:
print("=== VCUBE CLASS SUMMARY ===")
print()
print("🧩 CORE CONCEPT:")
print("• VCube represents a virtual 3x3 Rubik's cube")
print("• State stored as 54-character string (6 faces × 9 facelets)")
print("• Face order: U(Up), R(Right), F(Front), D(Down), L(Left), B(Back)")
print("• Each face: 9 facelets numbered 0-8 (top-left to bottom-right)")
print("• Solved state: each face has uniform color (UUUUUUUUU, etc.)")
print()
print("🏗️ CREATION METHODS:")
print("• VCube() - default solved cube")
print("• VCube(state_string) - from specific 54-character state")
print("• VCube.from_cubies(cp, co, ep, eo, so) - from cubie representation")
print("• cube.copy() / cube.copy(full=True) - create copies")
print("• cube.oriented_copy(orientation) - copy with specific orientation")
print()
print("🔄 MOVE APPLICATION:")
print("• rotate(moves) - apply move sequence (string or Algorithm)")
print("• rotate_move(move) - apply single move")
print("• Support for all standard notation: R, R', R2, Rw, M, x, y, z")
print("• History tracking (optional) for move sequences")
print("• Integration with Algorithm objects")
print()
print("📊 STATE ANALYSIS:")
print("• .state property - get current 54-character state")
print("• .is_solved property - check if cube is solved")
print("• .orientation property - current orientation (top-front faces)")
print("• get_face(face) - get specific face's 9 facelets")
print("• get_face_center_indexes() - get all face center colors")
print("• .to_cubies property - convert to cubie representation")
print()
print("🎨 VISUALIZATION:")
print("• show() method - print visual representation")
print("• display() method - return visual representation as string")
print("• Support for different modes, orientations, and palettes")
print("• Masking capabilities to highlight specific areas")
print("• __str__() method - text representation by face")
print()
print("🧭 ORIENTATION FEATURES:")
print("• Automatic orientation detection from face centers")
print("• compute_orientation_moves() - calculate rotation sequences")
print("• oriented_copy() - create reoriented copies")
print("• is_equal() with strict/flexible comparison modes")
print("• Support for orientation-independent analysis")
print()
print("✅ INTEGRITY CHECKING:")
print("• Comprehensive state validation (check=True by default)")
print("• Validates state length, color counts, and cube mathematics")
print("• Ensures cube states are solvable")
print("• Can be disabled (check=False) for performance")
print("• check_integrity() method for manual validation")
print()
print("🔧 PERFORMANCE FEATURES:")
print("• Efficient C extension for move application")
print("• Optional integrity checking for trusted scenarios")
print("• Optional history tracking for memory efficiency")
print("• Lightweight copy operations")
print("• Fast state comparison and analysis")
print()
print("🎯 PRACTICAL APPLICATIONS:")
print()
print("FOR SPEEDCUBERS:")
print("• Practice algorithm effects without physical cube")
print("• Analyze algorithm cycles and patterns")
print("• Compare different algorithm approaches")
print("• Visualize cube states and transformations")
print("• Study orientation-independent solving")
print()
print("FOR DEVELOPERS:")
print("• Virtual cube simulation engine")
print("• Algorithm testing and validation")
print("• Cube state analysis and comparison")
print("• Educational applications and visualizations")
print("• Cube solving algorithm development")
print("• Performance analysis and optimization")
print()
print("⚡ INTEGRATION FEATURES:")
print("• Seamless Algorithm object support")
print("• VCubeDisplay integration for rich visualization")
print("• VCubeIntegrityChecker for state validation")
print("• Extension system for custom move implementations")
print("• Cubie/facelet conversion utilities")
print()
print("🛡️ ERROR HANDLING:")
print("• Comprehensive input validation")
print("• Clear error messages for debugging")
print("• Graceful handling of edge cases")
print("• Safe operation patterns")
print("• State corruption detection")
print()
print("💡 BEST PRACTICES:")
print("• Use integrity checking for external/user data")
print("• Disable checking (check=False) for trusted/performance scenarios")
print("• Use copy() for independent cube instances")
print("• Leverage orientation features for flexible analysis")
print("• Handle errors gracefully in production applications")
print("• Use appropriate copy modes (basic vs full) based on needs")
print()

# Final comprehensive example
print("🎯 COMPREHENSIVE EXAMPLE:")
print("="*30)

# Create and manipulate cube
demo_cube = VCube()
print(f"1. Created solved cube: {demo_cube.is_solved}")

# Apply algorithm
algorithm = Algorithm.parse_moves("R U R' F' R U R' U' R' F R2 U' R'")
demo_cube.rotate(algorithm)
print(f"2. Applied T-Perm: {demo_cube.is_solved}")

# Analyze state
print(f"3. State: {demo_cube.state[:20]}...")
print(f"4. Orientation: {demo_cube.orientation}")
print(f"5. History length: {len(demo_cube.history)}")

# Create oriented copy
try:
    oriented = demo_cube.oriented_copy("UR")
    print(f"6. Oriented copy: {oriented.orientation}")
    flexible_equal = demo_cube.is_equal(oriented, strict=False)
    print(f"7. Equivalent under orientation: {flexible_equal}")
except Exception as e:
    print(f"6-7. Orientation operations: {e}")

# Test cycle behavior
test_cube = VCube()
test_cube.rotate(algorithm)
print(f"8. T-Perm is self-inverse: {test_cube.is_solved}")

print("\n" + "="*60)
print("VCube: The Foundation of Virtual Cube Simulation! 🧩")
print("="*60)

=== VCUBE CLASS SUMMARY ===

🧩 CORE CONCEPT:
• VCube represents a virtual 3x3 Rubik's cube
• State stored as 54-character string (6 faces × 9 facelets)
• Face order: U(Up), R(Right), F(Front), D(Down), L(Left), B(Back)
• Each face: 9 facelets numbered 0-8 (top-left to bottom-right)
• Solved state: each face has uniform color (UUUUUUUUU, etc.)

🏗️ CREATION METHODS:
• VCube() - default solved cube
• VCube(state_string) - from specific 54-character state
• VCube.from_cubies(cp, co, ep, eo, so) - from cubie representation
• cube.copy() / cube.copy(full=True) - create copies
• cube.oriented_copy(orientation) - copy with specific orientation

🔄 MOVE APPLICATION:
• rotate(moves) - apply move sequence (string or Algorithm)
• rotate_move(move) - apply single move
• Support for all standard notation: R, R', R2, Rw, M, x, y, z
• History tracking (optional) for move sequences
• Integration with Algorithm objects

📊 STATE ANALYSIS:
• .state property - get current 54-character state
• .is_solved pro